In [244]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [245]:
def make_pred(x, w, b):
    return np.matmul(x, w) + b

def compute_cost(x, y, w, b):
    sqr_diff = (make_pred(x, w, b) - y)**2
    return sqr_diff.sum() / (2*x.shape[0])

def run_grad_decent(x, y, w, b, alpha):
    #start_time= time.time()
    cost_sum = ((make_pred(x, w, b)-y).sum() / x.shape[1])
    new_b = b - alpha * cost_sum

    j = x.shape[1]
    i = x.shape[0]
    new_w = np.empty(j)
    #start_time_1= time.time()
    summ = np.matmul(x, w)+b-y
    print("summ:", summ)
    summ_2 = x*summ
    print("summ_2:", summ_2 )
    # summ_2 = summ[k]*x[k]
    for k in range(i):
        #start_time_3= time.time()
        print(summ_2[k])
        print((summ_2[k]).reshape(-1))
        new_w += (summ_2[k]).reshape(-1)
        '''
        if not (test2 == new_w).all():
            print(test2)
            print(new_w)
            assert False
        '''
        #end_time_3= time.time()
        #print("run_grad_decent sub func make pred time:", (end_time_3-start_time_3)/(end_time_2-start_time_2))
    #end_time_1= time.time()
    #print("run_grad_decent sub func time:", end_time_1-start_time_1)
    new_w /= i
    new_w = w - (alpha * new_w.reshape([j, 1]))

    #end_time= time.time()
    #print("run_grad_decent func time:", end_time-start_time)
    return new_w, new_b

def run_linear_regression(x, y, alpha = .002, iter = 1000):    
    j = x.shape[1]
    w = np.zeros([j, 1])
    b = 0
    cost_history = []
    #print("-------------")
    print(w,b)
    #start_time= time.time()
    print(compute_cost(x, y, w, b))
    cost_history.append(compute_cost(x, y, w, b))
    #end_time= time.time()
    #print("time to compute cost once: ", end_time-start_time)
    for _ in range(iter):
        #start_time= time.time()
        w, b = run_grad_decent(x, y, w, b, alpha)
        #end_time= time.time()
        #print("time to run run_grad_decent once: ", end_time-start_time)
        cost_history.append(compute_cost(x, y, w, b))
    print(compute_cost(x, y, w, b))
    return w, b, cost_history 

In [246]:
# Grab dataset from file
car_dataframe = pd.read_csv("data/car details v4.csv")
data = (car_dataframe.loc[:, ["Fuel Tank Capacity", "Year", "Price"]]).to_numpy().astype('float64')

# Use simple data cleaning of dropping all null data
data_clean = data[~np.isnan(data).any(axis=1)]

# Suffle data insure equal distabution between train, validation, and test sets (Also use seed to insure determinisum)
rng = np.random.default_rng(seed=42)
rng.shuffle(data_clean)

# Split up data set into train, validation, and test
train_set_end_index = int(data_clean.shape[0]/2)
validation_set_end_index = int(3*data_clean.shape[0]/4)
train_set = data_clean[:train_set_end_index]
validation_set = data_clean[train_set_end_index:validation_set_end_index]
test_set = data_clean[validation_set_end_index:]

x = train_set[:, :2]
y = train_set[:, 2:]

normalization_vars = np.empty((x.shape[1], 2))

for i in range(x.shape[1]):
    mean = x[:, i:i+1].mean()
    std_dev = x[:, i:i+1].std()
    x[:, i:i+1] = (x[:, i:i+1] - mean) / std_dev
    normalization_vars[i] = [mean, std_dev]

# Normalize targets to prevent overflow error.
target_mean = y.mean()
y = y / target_mean
# run_linear_regression(x, y)

polynomial_architectures = [x]
validation_set_architectures = [x]

polynomial_degree = 3

for _i in range(polynomial_degree-1):
    last_polygon = polynomial_architectures[-1]
    polynomal_set = np.empty([last_polygon.shape[0], last_polygon.shape[1]**2])

    for i in range(last_polygon.shape[0]):
        index = 0
        for j in range(last_polygon.shape[1]):
            for k in range(last_polygon.shape[1]):
                polynomal_set[i, index] = last_polygon[i, j] * last_polygon[i, k]
                index += 1

    polynomial_architectures.append(polynomal_set)

for _i in range(polynomial_degree-1):
    last_polygon = validation_set_architectures[-1]
    polynomal_set = np.empty([last_polygon.shape[0], last_polygon.shape[1]**2])

    for i in range(last_polygon.shape[0]):
        index = 0
        for j in range(last_polygon.shape[1]):
            for k in range(last_polygon.shape[1]):
                polynomal_set[i, index] = last_polygon[i, j] * last_polygon[i, k]
                index += 1

    validation_set_architectures.append(polynomal_set)


for x in polynomial_architectures:
    for i in range(x.shape[1]):
        mean = x[:, i:i+1].mean()
        std_dev = x[:, i:i+1].std()
        x[:, i:i+1] = (x[:, i:i+1] - mean) / std_dev

cost_histories = []

for i in range(polynomial_degree):
    print(polynomial_architectures[i])
    _,_, cost_history = run_linear_regression(polynomial_architectures[i], y)
    cost_histories.append(cost_history)

from itertools import cycle

cycol = cycle('bgrcmk')
for i in cost_histories:
    plt.plot(i, color = next(cycol))
    plt.show()

for architecture in polynomial_architectures:
    # run_linear_regression(architecture, y)
    # Get train set x and y
    # Get validation set x and y
    # Train on train set
    # Save cost of traing set
    # Save cost of validation set
    pass

[[-1.15184597 -1.72930087]
 [-0.68318066 -0.50669615]
 [ 0.92367182  0.10460621]
 ...
 [ 0.18719776  1.63286211]
 [ 0.92367182 -0.50669615]
 [ 1.19148056  0.10460621]]
[[0.]
 [0.]] 0
1.4051423544996315
summ: [[ -0.14438087]
 [ -0.28587413]
 [ -2.42559865]
 [ -4.49313275]
 [ -0.13109783]
 [ -0.39849121]
 [ -0.53998446]
 [ -0.53420923]
 [ -0.15015611]
 [ -0.33207601]
 [ -2.18015117]
 [ -0.14726791]
 [ -0.40368892]
 [ -0.24544748]
 [ -0.32630077]
 [ -0.37539027]
 [ -2.02133221]
 [ -2.28121778]
 [ -0.47356868]
 [ -0.24544748]
 [ -0.16748181]
 [ -0.16459419]
 [ -0.08951614]
 [ -0.46201879]
 [ -0.30319983]
 [ -0.39849121]
 [ -0.22523416]
 [ -0.77965671]
 [ -0.99853811]
 [ -0.24544748]
 [ -0.31763792]
 [ -3.40738859]
 [ -0.92114997]
 [ -1.26824158]
 [ -0.49955782]
 [ -0.08374091]
 [ -0.35864209]
 [ -0.45913117]
 [ -1.60262768]
 [ -0.430255  ]
 [ -0.21657131]
 [ -1.03954228]
 [ -0.17267952]
 [ -1.25611359]
 [ -0.51399591]
 [ -0.15015611]
 [ -3.75390268]
 [ -0.54864731]
 [ -0.43314262]
 [ -0.22

AssertionError: 